In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def NS_users_crawler(codes, page):
    # User-Agent 설정
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}
    result_df = pd.DataFrame([])

    n_ = 0
    for page in range(1, page):
        n_ += 1
        if (n_ % 10 == 0):
            print('================== Page ' + str(page) + ' is done ==================')
        url = "https://finance.naver.com/item/board.naver?code=%s&page=%s" % (codes, str(page))
        # html → parsing
        html = requests.get(url, headers=headers).content
        # 한글 깨짐 방지 decode
        soup = BeautifulSoup(html.decode('euc-kr', 'replace'), 'html.parser')
        table = soup.find('table', {'class': 'type2'})
        tb = table.select('tbody > tr')

        for i in range(2, len(tb)):
            if len(tb[i].select('td > span')) > 0:
                date = tb[i].select('td > span')[0].text
                title = tb[i].select('td.title > a')[0]['title']
                views = tb[i].select('td > span')[1].text
                pos = tb[i].select('td > strong')[0].text
                neg = tb[i].select('td > strong')[1].text
                table = pd.DataFrame({'날짜': [date], '제목': [title], '조회': [views], '공감': [pos], '비공감': [neg]})
                result_df = pd.concat([result_df, table], ignore_index=True)

    return result_df

data = NS_users_crawler("066970", 3)
data.head(10)

,날짜,제목,조회,공감,비공감
0,2023.08.25 12:13,■■ 사악한 한투와 KB증권....,36,1,0
1,2023.08.25 12:13,박순혁작가께서...........,64,1,7
2,2023.08.25 12:12,월욜에도,42,2,1
3,2023.08.25 12:10,밧데리 .. 인버스 ..,49,2,0
4,2023.08.25 12:10,개미분들~~,89,2,2
5,2023.08.25 12:09,"포홀,프로,비엠 주주입니다",93,1,0
6,2023.08.25 12:08,시마이다,50,0,1
7,2023.08.25 12:07,실시간 정보.,68,1,2
8,2023.08.25 12:05,돔황챠..,88,3,1
9,2023.08.25 12:04,"초단기고점 280,000원",92,4,1
